In [1]:
import numpy as np
import sys
sys.path.append('/Users/zhengj10/Python/Modules')
import PatcherBot as pb
import pandas as pd
from glob import glob
import h5py
import os
import matplotlib.pyplot as plt


In [ ]:
# df = pd.read_pickle('/Users/zhengj10/Desktop/Frames drop issue/frames_drops.pkl')

In [ ]:
# df.head()

In [ ]:
# ax = df.stim_idx.plot(kind='hist',bins=20)
# # ax.set_xlabel('No. of frames drop')
# # ax.set_ylabel('Frequency')
# plt.show()

In [ ]:
# def get_peak_idx(trace,samples_per_ms = 40.0):
#     peak_indices = np.where(trace==1) # All indices with value equal to 1 (high)
#     peak_index = [k for k in range(peak_indices[0].shape[0]) if peak_indices[0][k]-peak_indices[0][k-1]!=1]
#     # Note the first peak was counted in different way: it's counted because it's subtracting the last value
#     # in the list and the result is not equal to 1.
#     peak_index = peak_indices[0][peak_index]
#     # peak_indices[0]: remove tuple format
#     peak_index = peak_index/samples_per_ms
#     for idx in range(peak_index.shape[0]):
#         if peak_index[idx]>=500.0:
#             break
#         else:
#             continue
#     stim_timing = peak_index[idx]
#     return peak_index,idx,stim_timing

In [3]:
imgPath = '/Volumes/genie/BenArthur/patch-screen/bot'
rawPath = '/Volumes/genie/Jihong_GENIE_stuff'

# imgPath = '/Volumes/Backup Plus/patch-screen/bot'
# rawPath = '/Volumes/Backup Plus/PatcherBot/raw'

In [5]:
date_path = ['2020.02.02_000']
# date_path = pb.getPathInfo(imgPath)
datePath = []
for date in date_path:
    if len(date)!=14:
        continue
    datePath.append(date)

In [6]:
datePath

['2020.02.02_000']

In [7]:
# datePath = pb.getPathInfo(imgPath)
no_cells = 0
frame_offset_all_cells = []
idx_all_cells = []
timing_all_cells = []
traces = []
cell_id = []
for date in datePath:
#     if len(date)!=14:
    #     continue
    platePath = pb.getPathInfo(os.path.join(imgPath, date))
    for plate in platePath:
        cellPath = pb.getPathInfo(os.path.join(imgPath, date, plate))
        for cell in(cellPath):
            print(cell)
            if not os.path.exists(os.path.join(rawPath, date, plate, cell)):
                continue
            matFiles = glob(os.path.join(imgPath, date, plate, cell) + '/*.mat')
            if len(matFiles) != 9:
                continue
            idx_all_trials = []
            timing_all_trials = []
            frame_offset_all_trials = []
#             for trial in range(9):
            daqData = os.path.join(rawPath,date,plate,cell,'camera_clamp2_000','000','Camera','daqResult.ma')
            imgData = os.path.join(rawPath,date,plate,cell,'camera_clamp2_000','000','Camera','frames.ma')
            if not os.path.exists(daqData):
                daqData = os.path.join(rawPath,date,plate,cell,'camera_clamp1_000','000','Camera','daqResult.ma')
            if not os.path.exists(imgData):
                imgData = os.path.join(rawPath,date,plate,cell,'camera_clamp1_000','000','Camera','frames.ma')
            daqData = h5py.File(daqData,'r')
            V_SYNC = daqData['data'][()][1]
            if np.max(V_SYNC)<1:
                print(os.path.join(rawPath, date, plate, cell)+'no vsync signal')
                continue

            peak_indices = np.where(V_SYNC==1)
            peak_index = [k for k in range(peak_indices[0].shape[0]) if peak_indices[0][k]-peak_indices[0][k-1]!=1]
            peak_index = peak_indices[0][peak_index]
            peak_index = peak_index/40.0 # change to ms
#             print(peak_index.shape)
            for i in range(peak_index.shape[0]):
                if peak_index[i]>=500.0:
#                     print(i,peak_index[i])
                    idx = i
                    break

            stim_timing = peak_index[idx]
            idx_all_trials.append(idx)

            timing_all_trials.append(stim_timing)
            imgData = h5py.File(imgData,'r')
            imgData = imgData['data'][()]

            F_all = []
            for frame in range(imgData.shape[0]):
                F = np.mean(imgData[frame,240:272,:])
                F_all.append(F)
            for frame in range(len(F_all[idx-20:idx+20])-3):
                if np.abs(F_all[idx-20:idx+20][frame+3]-F_all[idx-20:idx+20][frame])>3:
                    F_change_idx = frame+1
                    break
            frame_offset = 20-F_change_idx
            frame_offset_all_trials.append(frame_offset)
            frame_offset_all_cells.append(frame_offset_all_trials)
            idx_all_cells.append(idx_all_trials)
            timing_all_cells.append(timing_all_trials)
            traces.append(F_all[idx-20:idx+20])
            
            no_cells += 1
            cell_id.append(date+'/'+plate+'/'+cell)
        print(os.path.join(rawPath, date, plate))

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/genie/BenArthur/patch-screen/bot/2020.02.02_000'

In [ ]:
# plt.plot(F_all[idx-30:idx+10])

In [ ]:
no_cells

In [ ]:
# frame_offset_all_cells
# len(idx_all_cells)
# len(timing_all_cells)

In [ ]:
frame_offset = []
for i in range(len(frame_offset_all_cells)):
    frame_offset.append(frame_offset_all_cells[i][0])


In [ ]:
stim_idx = []
for i in range(len(idx_all_cells)):
    stim_idx.append(idx_all_cells[i][0])
    

In [ ]:
stim_timing = []
for i in range(len(timing_all_cells)):
    stim_timing.append(timing_all_cells[i][0])
    

In [ ]:
x = np.arange(no_cells)
plt.scatter(x,frame_offset)
# plt.ylim([479,485])
# plt.xlim([500,600])
# plt.savefig('/Users/zhengj10/Desktop/frames_offset_individual.png')
plt.show()

In [ ]:
np.median(frame_offset)

In [ ]:
plt.plot(F_all[idx-20:idx+20])
# plt.ylim([479,485])
# plt.xlim([2,10])


In [ ]:
plt.plot(stim_idx)
plt.ylim([470,494])
plt.tight_layout()
# plt.savefig('/Users/zhengj10/Desktop/stim_idx.png')
plt.show()

In [ ]:
plt.plot(stim_timing)
# plt.ylim([500.5,500.9])
plt.tight_layout()
# plt.savefig('/Users/zhengj10/Desktop/frames_offset.png')
plt.show()

In [ ]:
fig = plt.figure()
traces = df.traces.to_list()
x = np.linspace(-20,19,40)
for trace in traces:
    trace = np.array(trace)
    plt.plot(x,trace-trace[0])
plt.xlabel('frame#')
plt.ylabel('deltaF')
plt.tight_layout()
# fig.savefig('/Users/zhengj10/Desktop/Frames drop issue/traces.png',dpi=1200)


In [ ]:
df = pd.DataFrame({'cell_id':cell_id,'frame_offset':frame_offset,'stim_idx':stim_idx,'stim_timing':stim_idx,'traces':traces})

In [ ]:
df

In [ ]:
# df.to_pickle('/Users/zhengj10/Desktop/frames_drops.pkl')
# df.to_excel('/Users/zhengj10/Desktop/frames_drops.xlsx')

In [ ]:
ax = df.frame_offset.plot(kind='hist')
ax.set_xlabel('No. of frames offset')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
ax = df.frame_offset.plot()
ax.set_xlabel('Cell #')
ax.set_ylabel('No. of frames offset')
plt.show()

In [ ]:
dataPath = '/Volumes/Backup Plus/PatcherBot/raw/2020.10.15_000/plate_000/patch_attempt_0000_000/camera_clamp1_000/008'

In [ ]:
# dataFile = dataPath+'/Clamp1.ma'
# dataFile = dataPath+'Camera/daqResult.ma'
dataFile = dataPath+'/Camera/frames.ma'

In [ ]:
data = h5py.File(dataFile,'r')

In [ ]:
# data = data['data'][()]
data = data['info']['0']['values'][()]

In [ ]:
data.shape

In [ ]:
plt.plot(data[0])
plt.xlim([19990,20010])

In [ ]:
F_all = []
for frame in range(data.shape[0]):
    F = np.mean(data[frame,234:274,:])
    F_all.append(F)

In [ ]:
plt.plot(F_all)
plt.xlim([460,500])

In [ ]:
timing_info = []
for timing in range(data.shape[0]-1):
    timing_info.append(data[timing+1]-data[timing])

plt.plot(timing_info)

In [ ]:
class student:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def double_it(self,x):
        return x*2
    def get_mean(self,x):
        return np.mean(x)

        

In [ ]:
jihong = student('jihong',22)

In [ ]:
jihong.age

In [ ]:
jihong.double_it(10)

In [ ]:
jihong.get_mean([2,3])